# Introduzione


# Logistica dell'ultimo miglio

Con logistica dell'ultimo miglio si intende la parte del processo logistico che consegna la merce dal magazzino locale al cliente. Siamo quindi in una realtà dominata dagli imballi più disparati e da una pluralità di veicoli, spesso anche con esigenze diverse.

# Librerie
Il mio lavoro si concentra sullo sviluppo della libreria *py3dbl*, come alternativa/sostituto alla librearia *py3dbp*, allo scopo di migliorare, risolvendo i bug incontrati, e potenziare i meccanismi per la risoluzione del 3D bin packing ho sviluppato la seguente libreria:

In [ ]:
import py3dbl

Come sistema numerico usiamo la classe D, che permette operazioni in matematica esatta:

In [ ]:
from decimal import Decimal as D

# Elementi per casi di studio

## Mezzi
Per eseguire esperimenti su un caso realistico dobbiamo considerare dei mezzi atti allo scopo della logistica dell'ultimo miglio. In generale tutti i mezzi saranno soggetti ai seguenti vincoli:

In [ ]:
vincoli_base = [
    py3dbl.constraints['weight_within_limit'],
    py3dbl.constraints['fits_inside_bin'],
    py3dbl.constraints['no_overlap'],
    py3dbl.constraints['is_supported'],
]

### Furgone _- Fiat Ducato (L2H2)_

Un mezzo da carico piuttosto comune nell'ambiente della logistica dell'ultimo miglio sono i classici furgoni. Per avere un modello di riferimento ho scelto di usare un Ducato della Fiat, di cui sono riuscito ad avere misure e dettagli.

Lo spazio del vano di carico ha le seguenti dimensioni:

In [ ]:
dimensioni = py3dbl.Vector3(x = 1.87, y = 1.932, z = 3.120)

Come si vede però da questa immagine dobbiamo considerare gli ingombri dovuti ai sopra ruote:

![](./DucatoRear.jpg)

Anche se non ho misure certe nella profondità, consideriamo i seguenti modelli a 3 approssimazioni:

In [ ]:
sopra_ruota_sinistro = [
    py3dbl.Volume([D(.024),D(.4),D(.5)],[0,0,D(2.30)]),
    py3dbl.Volume([D(.15),D(.38),D(.48)],[D(.024),0,D(2.32)]),
    py3dbl.Volume([D(.05),D(.36),D(.47)],[D(.024)+D(.15),0,D(2.33)])
]
sopra_ruota_destro = [
    py3dbl.Volume([D(.024),D(.4),D(.5)],[D(1.87)-D(.024),0,D(2.30)]),
    py3dbl.Volume([D(.15),D(.38),D(.48)],[D(1.87)-D(.15)-D(.024),0,D(2.32)]),
    py3dbl.Volume([D(.05),D(.36),D(.47)],[D(1.87)-D(.05)-D(.15)-D(.024),0,D(2.33)])
]

Dobbiamo però aggiungere un vincolo speciale visto che non è possibile mettere oggetti che non passano dalla strettura oltre di essa:

In [ ]:
@py3dbl.constraint(weight=11)
def can_pass_over_the_upper_wheel(bin : py3dbl.Bin, item : py3dbl.Item):
    return item.position.z > D(2.8) or (item.width < D(1.442) or item.height < D(1.432))

A questo punto siamo pronti per costruire il modello:

In [ ]:
DucatoL2H2 = py3dbl.BinModel(
    name = "DucatoL2H2",
    size = dimensioni,
    max_weight = 2500,
    constraints = [*vincoli_base,py3dbl.constraints['can_pass_over_the_upper_wheel']],
    dead_volumes = [
        *sopra_ruota_sinistro, # vanno passati i singoli volumi
        *sopra_ruota_destro    # quindi li devo "spacchettare"
    ]
)

Visualizziamo quindi il risultato:

In [ ]:
py3dbl.render_bin_interactive(py3dbl.Bin(id=None,model=DucatoL2H2),render_bin=True)

### Porta Pacchi da Scooter _- Piaggio 3W - Delivery_

Consideriamo ora un mezzo particolare come il porta pacchi di uno scooter. Per avere un riferimento ho considerato il modello Delivery dei 3 ruote Piaggio, uno dei classici mezzi dei postini italiani dal 2021. Non avendo a disposizione dati riguardanti le misure del vano ho ipotizzato le seguenti dimensioni massime:

In [ ]:
dimensioni = py3dbl.Vector3(x = .79, y = .75, z = .71)

Adesso dobbiamo modellare tutte le forme del vano ritagliandole dalle dimensioni, come prima ipotizziamo delle misure verosimili:

In [ ]:
w,h,d = dimensioni
dead_volumes = [
    # triangolino alto
    py3dbl.Volume([w,D(.1),D(.02)],[0,h-D(.1),0]),
    py3dbl.Volume([w,D(.05),D(.1)],[0,h-D(.05),D(.02)]),
    # slitta alta
    py3dbl.Volume([w,D(.02),d-D(.13)],[0,h-D(.02),D(.13)]),
    py3dbl.Volume([w,D(.05),d-D(.13)-D(.05)],[0,h-D(.02)-D(.05),D(.13)+D(.05)]),
    py3dbl.Volume([w,D(.05),d-D(.13)-D(.05)-D(.1)],[0,h-D(.02)-D(.05)-D(.05),D(.13)+D(.05)+D(.1)]),
    # slitta retro
    py3dbl.Volume([w,h-D(.12)-D(.13),D(.05)],[0,D(.13),d-D(.05)]),
    py3dbl.Volume([w,h-D(.12)-D(.13)-D(.2),D(.08)],[0,D(.13)+D(.2),d-D(.05)-D(.08)]),
    py3dbl.Volume([w,h-D(.12)-D(.13)-D(.2)-D(.2),D(.08)],[0,D(.13)+D(.2)+D(.2),d-D(.05)-D(.08)-D(.08)]),
    #triangolino basso
    py3dbl.Volume([w,D(.04),D(.12)],[0,0,d-D(.12)]),
    py3dbl.Volume([w,D(.04)+D(.04),D(.06)],[0,D(.04),d-D(.06)])
]

A questo punto creiamo il modello e vediamo:

In [ ]:
Delivery = py3dbl.BinModel(
    name = "Delivery",
    size = dimensioni,
    max_weight = 60,
    constraints = vincoli_base,
    dead_volumes = dead_volumes
)
py3dbl.render_bin_interactive(py3dbl.Bin(None,Delivery))

## Creare una flotta

Consideriamo di avere una flotta di 2 Delivery e 1 Ducato e di dare identificativi semplici come S più un numero per gli scooter e V più numero per i furgoni, ci basta eseguire:

In [ ]:
n_s, n_v = 2, 1
fleet = [
    *[py3dbl.Bin(id=f"S{idx}",model=Delivery)   for idx in range(n_s)],
    *[py3dbl.Bin(id=f"V{idx}",model=DucatoL2H2) for idx in range(n_v)],
]

## Ordini

Generiamo una lista di ordini casuale usando il metodo di supporto item_generator:

In [ ]:
orders = py3dbl.item_generator(
    width  = (0.05,1.5),
    height = (0.05,1.5),
    depth  = (0.05,1.5),
    weight = (0.1,99),
    priority_range = (0,10),
    batch_size   = 40,
    use_gaussian_distrib = False,
    decimals = 3
)

# Il Risolutore

Ora che abbiamo mezzi e ordini non ci resta che cercare una soluzione. La libreria mette a disposizione la classe Packer che permette di creare una configurazione per la risoluzione.

In [ ]:
packer = py3dbl.Packer()
packer.add_fleet(fleet)  # aggiungo la flotta
packer.add_batch(orders) # Aggiungo la lista degli ordini

Prima della risoluzione possiamo settare parametri dei vincoli (e degli algoritmi) in base alle necessità.

In [ ]:
py3dbl.constraints['is_supported'].set_parameter("allow_item_fall",True)
py3dbl.constraints['is_supported'].set_parameter("minimum_support",0.9)

Lanciamo la risoluzione:

In [ ]:
import time
start = time.time()
packer.pack()
end = time.time()
print("Tempo d'esecuzione: ",end - start)
print("Bin usati: ",len(packer.current_configuration))

Possiamo vedere una statistica dei risultati principali usando *calculate_statistics*:

In [ ]:
for key,value in packer.calculate_statistics().items():
    print(str(key)+": "+str(round(value,2)))

Vediamo graficamente i risultati:

In [ ]:
for bin in packer.current_configuration:
    py3dbl.render_bin_interactive(bin)

## Altre configurazioni del risolutore

Il risolutore prevede diverse configurazioni che possono tornare utili, vediamo le principali:

Intanto rimuoviamo le configurazioni precedenti:

In [ ]:
packer.reset_fleet()
packer.reset_current_configuration()

Se si ha una flotta di veicoli tutti appartenenti allo stesso modello possiamo indicare un modello di default che il risolutore prenderà automaticamente:

In [ ]:
packer.set_default_bin(DucatoL2H2) # supponiamo una flotta di furgoni

Supponiamo di voler impostare l'algoritmo a *all_lay* con rotazioni, usando *set_algorithm* si setta l'algoritmo di default e si ottiene l'algoritmo precedentemente usato:

In [ ]:
py3dbl.algorithms['all_lay'].set_parameter("allow_full_rotation",True)
prev_algorithm = packer.set_algorithm(py3dbl.algorithms['all_lay'])
print(prev_algorithm.func.__name__)

Possiamo anche chiedere la risoluzione con un algoritmo passandolo al volo nel parametro *algorithm*:

In [ ]:
import time
start = time.time()
packer.pack(
    algorithm=py3dbl.algorithms['big_lay_small_stand']
) 
end = time.time()
print("Tempo d'esecuzione: ",end - start)
print("Bin usati: ",len(packer.current_configuration))

Vediamo quindi come è stato caricato il primo contenitore:

In [ ]:
py3dbl.render_bin_interactive(packer.current_configuration[0],render_bin=False,border_width=1.0,transparency=.8)